In [9]:
!pip install datasets evaluate

In [10]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
import evaluate
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [11]:
# 1. Load the IMDB dataset (binary sentiment classification)
dataset = load_dataset("imdb")
print(f"Dataset loaded with {len(dataset['train'])} training examples and {len(dataset['test'])} test examples")

# 2. Prepare tokenizer
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# 3. Define preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

# 4. Apply preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Dataset loaded with 25000 training examples and 25000 test examples


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:

# 5. Define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 6. Load pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)

# 7. Define training arguments
batch_size = 8
output_dir = "./bert-finetuned-imdb"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,  # Set to True if you want to upload to HF Hub
    report_to ="tensorboard"

)

# 8. Create data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 9. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 10. Train the model
print("Starting training...")
trainer.train()

# 11. Evaluate the model
print("Evaluating the model...")
evaluation_results = trainer.evaluate()
print(f"Evaluation results: {evaluation_results}")

# 12. Save the model locally
trainer.save_model(output_dir)
print(f"Model saved to {output_dir}")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-ecc66ee6818f>:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.257100,0.221795,0.930160,0.928302,0.953679,0.904240


In [ ]:
# 13. Example of how to use the fine-tuned model for inference
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        inputs = inputs.to(model.device)
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    confidence = probabilities[0][predicted_class].item()

    sentiment = "Positive" if predicted_class == 1 else "Negative"
    return {"sentiment": sentiment, "confidence": confidence}

# Test with some examples
test_texts = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "What a waste of time. The plot made no sense and the acting was terrible.",
    "It was okay, not great but not terrible either."
]

print("\nTesting the fine-tuned model:")
for text in test_texts:
    result = predict_sentiment(text)
    print(f"Text: {text}\nSentiment: {result['sentiment']} (Confidence: {result['confidence']:.4f})\n")



In [ ]:
# 14. How to load the model after saving
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("./bert-finetuned-imdb")
model = AutoModelForSequenceClassification.from_pretrained("./bert-finetuned-imdb")
